Philip McCormick

MSDS 598

February 28, 2022

Homework 2


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random

import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

df_taxis = sns.load_dataset('taxis')
df_taxis.corr()

,passengers,distance,fare,tip,tolls,total
passengers,1.000000,0.009411,0.007637,0.021099,-0.002903,0.015708
distance,0.009411,1.000000,0.920108,0.452589,0.635267,0.904676
fare,0.007637,0.920108,1.000000,0.488612,0.609307,0.974358
tip,0.021099,0.452589,0.488612,1.000000,0.413619,0.646186
tolls,-0.002903,0.635267,0.609307,0.413619,1.000000,0.683142
total,0.015708,0.904676,0.974358,0.646186,0.683142,1.000000


In [2]:
df_taxis.head(1)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.6,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan


1. Suppose we want to use the variables distance and passengers to predict total which
represents the total fare.

(a) Use linear regression to create a model prediction total from distance and passengers.
Report the R2.

In [3]:
model = ols(formula = 'total ~ distance + passengers', data=df_taxis)
res1 = model.fit()
print(f'This model has an R^2 of {res1.rsquared}')

This model has an R^2 of 0.8184911794925387


(b) Incorporate another variable taxi color into the model and then report the R2.

In [4]:
model = ols(formula = 'total ~ distance + passengers + color', data=df_taxis)
res2 = model.fit()
print(f'This model has an R^2 of {res2.rsquared}')

This model has an R^2 of 0.8286465295996958


2. As in the fourth lecture, create a length of ride variable. Use the new feature you’ve
engineered to further improve the model and report the R2.

In [16]:
pd.to_datetime(df_taxis['pickup'])

# time delta
pd.to_datetime(df_taxis['dropoff']) - pd.to_datetime(df_taxis['pickup'])

delta = pd.to_datetime(df_taxis['dropoff']) - pd.to_datetime(df_taxis['pickup'])
delta

# length of trip-converting to minutes
df_taxis['length_of_trip'] = delta / pd.Timedelta('60s')
df_taxis.head(1)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,length_of_trip
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.6,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan,6.25


In [17]:
model = ols(formula = 'total ~ distance + passengers + color + length_of_trip', data=df_taxis)
res3 = model.fit()
print(f'This model has an R^2 of {res3.rsquared}')

This model has an R^2 of 0.8468228892910934


3. Create a validation set using ten percent of the data, and use the complement to create
a training set. (Make sure there is no overlap in observations between the validation and
training set).

In [18]:
from sklearn.model_selection import train_test_split

# take random 10% of the taxis for testing
train, test = train_test_split(df_taxis, test_size=0.1)

In [20]:
# build model on training set
model = ols(formula = 'total ~ distance + passengers + color + length_of_trip', data=train)
res = model.fit()

# predict using the model
y_pred = res.predict(test)

# score the model
y = test['total']
y_mean = train['total'].mean()  # use the mean of the training set

TSS = sum((y - y_mean)**2)
RSS = sum((y_pred - y)**2)
print(f'This model has an R^2 on the test set of {(TSS - RSS) / TSS}')

This model has an R^2 on the test set of 0.902091461462861


4. Use this validation and the R2metric to decide whether adding length of ride to the
variables from Q1 improves the model.

Introducing more variables to the model will increase $R^2$ because more values added to the sum of squares will decrease variance. This does not mean the model is improved, it just increases $R^2$, which is often a temptation to add variables to strengthen the $R^2$ metric.

5. Engineer one new feature from the data and repeat Q4. (Hint: you can use powers,
differences, ratios, products of other features!)

In [ ]:
df_taxis.head(5)

In [ ]:
# new feature- set up dummy variable for payment type (cash/cc)
dummy = pd.get_dummies(df_taxis['payment'])

df2 = pd.concat([df_taxis, dummy], axis = 1)
df2.head(1)

In [27]:
from sklearn.model_selection import train_test_split

# take random 10% of the taxis for testing
train, test = train_test_split(df2, test_size=0.1)

In [28]:
# build model on training set
model = ols(formula = 'total ~ distance + passengers + color + cash', data=train)
res = model.fit()

# predict using the model
y_pred = res.predict(test)

# score the model
y = test['total']
y_mean = train['total'].mean()  # use the mean of the training set

TSS = sum((y - y_mean)**2)
RSS = sum((y_pred - y)**2)
print(f'This model has an R^2 on the test set of {(TSS - RSS) / TSS}')

This model has an R^2 on the test set of 0.8352464687988639


This feature of adding a dummy variable to determine if the rider payed with cash or not, did increase the value of $R^2$. This does not mean it added to the overall model, it simply increased $R^2$ since we added a new numeric variable to model which causes $R^2$ to go up.

6. Why shouldn’t you use fare or tip to predict total?

You should avoid using fare or tip to predict total because they are both directly related to the total. If the variables in the model are correlated to the dependant variable this will cause multicollinearity. Multicollinearity will cause leakage in the data set, and undermines the statistical significance of the variables. 